In [4]:
from math import *
import random

In [5]:
# In this exercise, try to write a program that
# will resample particles according to their weights.
# Particles with higher weights should be sampled
# more frequently (in proportion to their weight).

# Don't modify anything below. Please scroll to the 
# bottom to enter your code.

class robot:
    def __init__(self):
        self.x = random.random() * world_size
        self.y = random.random() * world_size
        self.orientation = random.random() * 2.0 * pi
        self.forward_noise = 0.0;
        self.turn_noise    = 0.0;
        self.sense_noise   = 0.0;
    
    def set(self, new_x, new_y, new_orientation):
        if new_x < 0 or new_x >= world_size:
            raise ValueError('X coordinate out of bound')
        if new_y < 0 or new_y >= world_size:
            raise ValueError('Y coordinate out of bound')
        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise ValueError('Orientation must be in [0..2pi]')
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)
    
    
    def set_noise(self, new_f_noise, new_t_noise, new_s_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.forward_noise = float(new_f_noise);
        self.turn_noise    = float(new_t_noise);
        self.sense_noise   = float(new_s_noise);
    
    
    def sense(self):
        Z = []
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            dist += random.gauss(0.0, self.sense_noise)
            Z.append(dist)
        return Z
    
    
    def move(self, turn, forward):
        if forward < 0:
            raise ValueError('Robot cant move backwards')       
        
        # turn, and add randomness to the turning command
        orientation = self.orientation + float(turn) + random.gauss(0.0, self.turn_noise)
        orientation %= 2 * pi
        
        # move, and add randomness to the motion command
        dist = float(forward) + random.gauss(0.0, self.forward_noise)
        x = self.x + (cos(orientation) * dist)
        y = self.y + (sin(orientation) * dist)
        x %= world_size    # cyclic truncate
        y %= world_size
        
        # set particle
        res = robot()
        res.set(x, y, orientation)
        res.set_noise(self.forward_noise, self.turn_noise, self.sense_noise)
        return res
    
    def Gaussian(self, mu, sigma, x):
        
        # calculates the probability of x for 1-dim Gaussian with mean mu and var. sigma
        return exp(- ((mu - x) ** 2) / (sigma ** 2) / 2.0) / sqrt(2.0 * pi * (sigma ** 2))
    
    
    def measurement_prob(self, measurement):
        
        # calculates how likely a measurement should be
        
        prob = 1.0;
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            prob *= self.Gaussian(dist, self.sense_noise, measurement[i])
        return prob
    
    def __repr__(self):
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))

In [6]:

def cdf(array):
    cumulative_dist = []
    cumulative_sum = 0
    for i in array:
        cumulative_sum += i
        cumulative_dist.append(cumulative_sum)
    
    return cumulative_dist

def binarySearch(val, array, start, end):
    mid_index = floor((end - start)/2) + start
    if start <= end:
        if(array[mid_index] == val):
            return mid_index
        elif (val > array[mid_index]):
            return binarySearch(val, array, mid_index+1, end)
        if (val < array[mid_index]):
            return binarySearch(val, array, start, mid_index-1)
    else:
        if(abs(array[mid_index] - val) > abs(array[mid_index + 1] - val)):
            return mid_index + 1
        else:
            return mid_index


def resample(array):
    cumulative = cdf(array)
    random_seed = random.choice(cumulative)
    return binarySearch(random_seed,cumulative,0,len(cumulative)-1)

In [7]:

def normalize(array):
    total = sum(array)
    for i in range(len(array)):
        array[i] = array[i]/total

    return array


def resampler(particles, weights):
    max_weight = max(weights)
    beta = 0
    p = []
    idx = []
    index = random.randint(0,len(weights) - 1)
    for i in range(len(weights)):        
        beta = beta + random.uniform(0, 2 * max_weight)
        while weights[index] < beta:
            beta = beta - weights[index]
            index = (index + 1) % len(weights) 
        p.append(particles[index])
        idx.append(index)
    #################################### Udacity code
    # index = int(random.random() * N)
    # beta = 0.0
    # mw = max(w)
    # for i in range(N):
    #     beta += random.random() * 2.0 * mw
    #     while beta > w[index]:
    #         beta -= w[index]
    #         index = (index + 1) % N
    #     p3.append(p[index])
    #####################################

    return p,idx

def eval(r, p):
    sum = 0.0;
    for i in range(len(p)): # calculate mean error
        dx = (p[i].x - r.x + (world_size/2.0)) % world_size - (world_size/2.0)
        dy = (p[i].y - r.y + (world_size/2.0)) % world_size - (world_size/2.0)
        err = sqrt(dx * dx + dy * dy)
        sum += err
    return sum / float(len(p))

In [11]:
landmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
world_size = 100.0

myrobot = robot()
myrobot = myrobot.move(0.1, 5.0)
Z = myrobot.sense()

N = 1000
iterations = 10
p = []
for i in range(N):
    x = robot()
    x.set_noise(0.05, 0.05, 5.0)
    p.append(x)

for t in range(iterations):

    myrobot = myrobot.move(0.1, 5.0)
    Z = myrobot.sense()
    p2 = []
    for i in range(N):
        p2.append(p[i].move(0.1, 5.0))
    p = p2

    w = []
    for i in range(N):
        w.append(p[i].measurement_prob(Z))


    #### DON'T MODIFY ANYTHING ABOVE HERE! ENTER CODE BELOW ####
    # You should make sure that p3 contains a list with particles
    # resampled according to their weights.
    # Also, DO NOT MODIFY p.
    w = normalize(w)
    p3 = resampler(p,w)

    p,idx_ = p3
    # print(len(p)) #please leave this print statement here for grading!

    error = eval(myrobot,p)
    print(error)


4.454613530447991
3.302531962226216
2.564885922588099
3.147025275557553
3.809614633456627
3.9578934647586617
4.168941616573897
4.22223250919116
4.158583221588213
3.8855109462601627
